In [23]:
import arff
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import TargetEncoder

from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

In [4]:
with open(r"../student-mental-health-analysis/data/raw/depression_dataset.arff", 'r') as f:
    dataset = arff.load(f)

In [5]:
df = pd.DataFrame(dataset['data'], columns=[a[0] for a in dataset['attributes']])

In [6]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2.0,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1.0
1,8.0,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0.0
2,26.0,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0.0
3,30.0,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1.0
4,32.0,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0.0


In [7]:
print("\nMissing values before handling:")
print(df.isnull().sum())


Missing values before handling:
id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         3
Family History of Mental Illness         0
Depression                               0
dtype: int64


In [8]:
num_cols = df.select_dtypes(include=[np.number]).columns
for col in num_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mean(), inplace=True)
        print(f"Filled missing values in '{col}' with MEAN")

Filled missing values in 'Financial Stress' with MEAN


C:\Users\msand\AppData\Local\Temp\ipykernel_13848\2339124779.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [9]:
cat_cols = df.select_dtypes(exclude=[np.number]).columns
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)
        print(f"Filled missing values in '{col}' with MODE")

In [10]:
print("\nMissing values after handling:")
print(df.isnull().sum())


Missing values after handling:
id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64


In [11]:
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])

In [12]:
te_city = TargetEncoder()
df['City'] = te_city.fit_transform(df['City'], df['Depression'])

In [13]:
prof_counts = df['Profession'].value_counts()

rare_prof = prof_counts[prof_counts < 10].index
df['Profession'] = LabelEncoder().fit_transform(df['Profession'].apply(lambda x: 'Other' if x in rare_prof else x))

In [14]:
df['Sleep Duration'] = df['Sleep Duration'].replace('Others', 'More than 8 hours')

sleep_order = ['Less than 5 hours', '5-6 hours', '7-8 hours', 'More than 8 hours']
ordinal_sleep = OrdinalEncoder(categories=[sleep_order])
df['Sleep Duration'] = ordinal_sleep.fit_transform(df[['Sleep Duration']])

In [15]:
df['Dietary Habits'] = df['Dietary Habits'].replace('Others', 'Moderate')

diet_order = ['Unhealthy', 'Moderate', 'Healthy']
ordinal_diet = OrdinalEncoder(categories=[diet_order])
df['Dietary Habits'] = ordinal_diet.fit_transform(df[['Dietary Habits']])

In [16]:
te_degree = TargetEncoder()
df['Degree'] = te_degree.fit_transform(df['Degree'], df['Depression'])

In [17]:
le_suicide = LabelEncoder()
df['Have you ever had suicidal thoughts ?'] = le_suicide.fit_transform(df['Have you ever had suicidal thoughts ?'])

In [18]:
le_family = LabelEncoder()
df['Family History of Mental Illness'] = le_family.fit_transform(df['Family History of Mental Illness'])

In [19]:
print(df.dtypes)

id                                       float64
Gender                                     int32
Age                                      float64
City                                     float64
Profession                                 int32
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                           float64
Dietary Habits                           float64
Degree                                   float64
Have you ever had suicidal thoughts ?      int32
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness           int32
Depression                               float64
dtype: object


In [20]:
df.head(5)

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2.0,1,33.0,0.569659,1,5.0,0.0,8.97,2.0,0.0,1.0,2.0,0.528395,1,3.0,1.0,0,1.0
1,8.0,0,24.0,0.608866,1,2.0,0.0,5.90,5.0,0.0,1.0,1.0,0.588964,0,3.0,2.0,1,0.0
2,26.0,1,31.0,0.556122,1,3.0,0.0,7.03,5.0,0.0,0.0,2.0,0.535000,0,9.0,1.0,1,0.0
3,30.0,0,28.0,0.531387,1,3.0,0.0,5.59,2.0,0.0,2.0,1.0,0.571528,1,4.0,5.0,1,1.0
4,32.0,0,25.0,0.524131,1,4.0,0.0,8.13,3.0,0.0,1.0,1.0,0.509785,1,1.0,1.0,0,0.0


In [ ]:
#outlier detection - shevon

In [21]:
numerical_cols = ['Age', 'Academic Pressure', 'Work Pressure', 'CGPA', 'Study Satisfaction',
                  'Job Satisfaction', 'Work/Study Hours', 'Financial Stress']

X_num = df[numerical_cols]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)

df_scaled = pd.DataFrame(X_scaled, columns=numerical_cols)

df[numerical_cols] = df_scaled

In [22]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2.0,1,1.463165,0.569659,1,1.345543,-0.009777,0.893393,-0.693425,-0.01534,1.0,2.0,0.528395,1,-1.121213,-1.488868,0,1.0
1,8.0,0,-0.371474,0.608866,1,-0.826104,-0.009777,-1.194076,1.510636,-0.01534,1.0,1.0,0.588964,0,-1.121213,-0.793092,1,0.0
2,26.0,1,1.055467,0.556122,1,-0.102222,-0.009777,-0.425724,1.510636,-0.01534,0.0,2.0,0.535000,0,0.497095,-1.488868,1,0.0
3,30.0,0,0.443921,0.531387,1,-0.102222,-0.009777,-1.404862,-0.693425,-0.01534,2.0,1.0,0.571528,1,-0.851495,1.294237,1,1.0
4,32.0,0,-0.167625,0.524131,1,0.621660,-0.009777,0.322229,0.041262,-0.01534,1.0,1.0,0.509785,1,-1.660650,-1.488868,0,0.0


In [24]:
df.drop(columns=['id'], inplace=True)

X = df.drop('Depression', axis=1)
y = df['Depression']

In [25]:
selector = SelectKBest(mutual_info_classif, k=10)
X_selected = selector.fit_transform(X, y)

selected_features = X.columns[selector.get_support()]
print("\nSelected top 10 features:")
print(selected_features)


Selected top 10 features:
Index(['Age', 'Academic Pressure', 'Study Satisfaction', 'Sleep Duration',
       'Dietary Habits', 'Degree', 'Have you ever had suicidal thoughts ?',
       'Work/Study Hours', 'Financial Stress',
       'Family History of Mental Illness'],
      dtype='object')


In [26]:
df = df[selected_features]
df['Depression'] = y 

In [27]:
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(df.drop('Depression', axis=1))

In [28]:
print("\nNumber of PCA components:", pca.n_components_)
print("Explained variance ratio:", pca.explained_variance_ratio_)


Number of PCA components: 8
Explained variance ratio: [0.18795819 0.16957945 0.13515293 0.12950508 0.12043142 0.11309521
 0.08320425 0.03375923]


In [29]:
df.head()

,Age,Academic Pressure,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,1.463165,1.345543,-0.693425,1.0,2.0,0.528395,1,-1.121213,-1.488868,0,1.0
1,-0.371474,-0.826104,1.510636,1.0,1.0,0.588964,0,-1.121213,-0.793092,1,0.0
2,1.055467,-0.102222,1.510636,0.0,2.0,0.535000,0,0.497095,-1.488868,1,0.0
3,0.443921,-0.102222,-0.693425,2.0,1.0,0.571528,1,-0.851495,1.294237,1,1.0
4,-0.167625,0.621660,0.041262,1.0,1.0,0.509785,1,-1.660650,-1.488868,0,0.0


In [ ]:
# Class balancing - dinith